In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
from tqdm import tqdm
from glob import glob
import tensorflow as tf
import librosa
import librosa.display as dsp
import IPython.display as ipd

warnings.filterwarnings(action='ignore')


In [11]:
sr = 20000

#시드 고정

import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything(813)

from tqdm.notebook import tqdm

def load_audio(file_names, path):   
  audios = []
  for audio in tqdm(file_names):
    # librosa를 이용하여 데이터 로드
    an_audio, _ = librosa.load(path+audio, sr=sr)
    audio_array = np.array(an_audio)
    audios.append(audio_array)
  audios = np.array(audios)


  return audios

In [9]:
arr=[]
for i in os.listdir('waaav'):
    if i[-1]=='v':
        arr.append(i)
arr

['Sess01_script01_User001F_001.wav',
 'Sess01_script01_User001F_002.wav',
 'Sess01_script01_User001F_003.wav',
 'Sess01_script01_User001F_004.wav',
 'Sess01_script01_User001F_005.wav',
 'Sess01_script01_User001F_006.wav',
 'Sess01_script01_User001F_007.wav',
 'Sess01_script01_User001F_008.wav',
 'Sess01_script01_User001F_009.wav',
 'Sess01_script01_User001F_010.wav',
 'Sess01_script01_User001F_011.wav',
 'Sess01_script01_User001F_012.wav',
 'Sess01_script01_User001F_013.wav',
 'Sess01_script01_User001F_014.wav',
 'Sess01_script01_User001F_015.wav',
 'Sess01_script01_User002M_001.wav',
 'Sess01_script01_User002M_002.wav',
 'Sess01_script01_User002M_003.wav',
 'Sess01_script01_User002M_004.wav',
 'Sess01_script01_User002M_005.wav',
 'Sess01_script01_User002M_006.wav',
 'Sess01_script01_User002M_007.wav',
 'Sess01_script01_User002M_008.wav',
 'Sess01_script01_User002M_009.wav',
 'Sess01_script01_User002M_010.wav',
 'Sess01_script01_User002M_011.wav',
 'Sess01_script01_User002M_012.wav',
 

In [13]:
audio_data = load_audio(arr, path='waaav/')

  0%|          | 0/13462 [00:00<?, ?it/s]

In [19]:
mfcc = librosa.feature.mfcc(y=audio_data[15], sr=16000, n_mfcc=50)
mfcc.shape

(50, 315)

In [22]:
size = 50
pad_size = 50
repeat_size = 5

In [34]:
import sklearn
from sklearn.preprocessing import StandardScaler
mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
mfcc.shape

(50, 315)

In [39]:
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
padded_mfcc = pad2d(mfcc, 600)

padded_mfcc.shape

(50, 600)

In [43]:
s=[]
p=[]

In [44]:
for i in range(13462):
    seg_id = arr[i][:-4]
    mfcc=librosa.feature.mfcc(y=audio_data[i], sr=16000, n_mfcc=50)
    mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
    padded_mfcc = pad2d(mfcc, 600)
    s.append(seg_id)
    p.append(padded_mfcc)
    

In [45]:
df=pd.DataFrame(zip(s, p))
df

,0,1
0,Sess01_script01_User001F_001,"[[-0.6640639901161194, 0.6258189678192139, 0.3..."
1,Sess01_script01_User001F_002,"[[-2.210876226425171, -1.751159906387329, -1.4..."
2,Sess01_script01_User001F_003,"[[-1.0509703, -0.6705827, -0.7513155, -0.90893..."
3,Sess01_script01_User001F_004,"[[-1.797121286392212, -1.4606393575668335, -1...."
4,Sess01_script01_User001F_005,"[[-1.8760236501693726, -1.672189712524414, -1...."
...,...,...
13457,Sess40_script06_User080F_021,"[[-1.7087924480438232, -1.5370320081710815, -1..."
13458,Sess40_script06_User080F_022,"[[-1.7637516260147095, -1.4955164194107056, -1..."
13459,Sess40_script06_User080F_023,"[[-1.4342446327209473, -1.172799825668335, -1...."
13460,Sess40_script06_User080F_024,"[[-1.3491223, -1.1192734, -1.1781569, -1.19942..."


In [46]:
#저장
import pickle

# 파일로 저장
with open('audio_mfcc_preprocessing.pkl', 'wb') as f:
    pickle.dump(df, f)